In [1]:
import random
import sage.graphs.graph_plot
from sage.graphs.base.boost_graph import *
from sage.plot.point import *
import numpy

In [2]:
#enako kot v "naivni_algoritem_koncni.ipynb"

def seznam_cen(st_opazanj, st_pov_grafa, min_cena, max_cena):
    sur_podatki = []
    for i in range(0, st_opazanj):
        mer_povezav = []
        for j in range(0, st_pov_grafa):
            mer_povezav.append(numpy.random.random_integers(min_cena, max_cena))
        sur_podatki.append(mer_povezav)
    return(sur_podatki)

#stevilo_pov_grafa = 2 * st_vozlisc^2 - 2 * st_vozlisc

In [3]:
#enako kot v "naivni_algoritem_koncni.ipynb"

def mnoz(vektor1, vektor2):
    matrika = []
    for i in range(0,len(vektor1)):
        vrstica =[]
        for j in range(0, len(vektor2)):
            vrstica.append(vektor1[i] * vektor2[j])
        matrika.append(vrstica)
    return(matrika)

def kov_matrika(vse_utezi):
    avr_pov = [i * 1/len(vse_utezi) for i in [sum(x) for x in zip(*vse_utezi)]]
    zac_mat = mnoz(numpy.subtract(vse_utezi[0],avr_pov),numpy.subtract(vse_utezi[0],avr_pov))
    for i in range(1, len(vse_utezi)):
        zac_mat = numpy.add(zac_mat, mnoz(numpy.subtract(vse_utezi[i],avr_pov),numpy.subtract(vse_utezi[i],avr_pov)))
    kon_mat = numpy.dot(1/len(vse_utezi), zac_mat)
    return(kon_mat)

In [4]:
#enako kot v "naivni_algoritem_koncni.ipynb"

def vektorcki_x(graf, zacetno, koncno):
    vse_poti = graf.all_paths(zacetno, koncno, use_multiedges=True, report_edges=False, labels=False)
    vse_povezave = graf.edges()
    par = []
    for i in range(0,len(vse_povezave)):
        par.append(vse_povezave[i][0:2])
    seznam_vektorjev = []
    for i in range(0, len(vse_poti)):
        pot = (vse_poti[i])
        prvi_oklepaj1 = list(zip(pot, pot[1:] + pot[:1]))
        prvi_oklepaj = prvi_oklepaj1[:-1]
        x=[]
        for i in range(0, len(par)):
            if par[i] in prvi_oklepaj:
                x.append(1)
            else:
                x.append(0)
        seznam_vektorjev.append(x)
    return(seznam_vektorjev)

In [5]:
#enako kot v "naivni_algoritem_koncni.ipynb"

def grid_graf(st_opazanj, st_vozlisc, min_cena, max_cena):
    st_pov_grafa = 2 * st_vozlisc^2 - 2 * st_vozlisc
    utezi = seznam_cen(st_opazanj, st_pov_grafa, min_cena, max_cena)
    graf_seznam = graphs.Grid2dGraph(st_vozlisc, st_vozlisc).edges()
    nov = [] #najprej spremeniva v list
    for i in range(0,len(graf_seznam)):
        nov.append(list(graf_seznam[i]))
    konec = [] #dodava še ceno, v obliki ki jo sprjema digraph
    seznam_grafov = []
    for j in range(0,len(utezi)):
        zadnji = []
        for i in range(0,len(nov)):
            vmesni = []
            vmesni.append(nov[i][0])
            vmesni.append(nov[i][1])
            vmesni.append(utezi[j][i])
            zadnji.append(tuple(vmesni))
        konec.append(zadnji)
        gr = DiGraph(zadnji)
        seznam_grafov.append(gr)
    return(seznam_grafov, utezi)


In [6]:
#enako kot v "naivni_algoritem_koncni.ipynb"

def min_grid_graf(st_opazanj, st_vozlisc, min_cena, max_cena):
    st_pov_grafa = 2 * st_vozlisc^2 - 2 * st_vozlisc
    utezi = seznam_cen(st_opazanj, st_pov_grafa, min_cena, max_cena)
    graf_oblika = DiGraph(graphs.Grid2dGraph(st_vozlisc, st_vozlisc).edges())
    return(graf_oblika, utezi)

In [7]:
#enako kot v "naivni_algoritem_koncni.ipynb"

def argmin(poti, av_pov, diagonala):
    x_min = poti[0]
    cena_pov = numpy.dot(av_pov, x_min)
    cena_d = numpy.dot(diagonala, x_min)
    cena = cena_pov + cena_d
    for i in range(1, len(poti)):
        x_nov = poti[i]
        cena_pov_nov = numpy.dot(av_pov, x_nov)
        cena_d_nov = numpy.dot(diagonala, x_nov)
        cena_nov = cena_pov_nov + cena_d_nov
        if cena_nov < cena:
            x_min = x_nov
            cena_pov = cena_pov_nov
            cena_d = cena_d_nov
            cena = cena_nov
    return(x_min, cena_pov, cena_d, cena)

In [8]:
#enako kot v "naivni_algoritem_koncni.ipynb"

def arglexmin(seznam_x, vektorji):
    xmin = min(([numpy.dot(numpy.array(v), numpy.array(x)) for v in vektorji], x) for x in seznam_x)
    return xmin

In [9]:
#rešuje optimizacijski problem najkrajše poti, kjer je skalarni produkt po diagonali matrike korenjen

def argmin_kor(poti, av_pov, diagonala):
    x_rob = poti[0]
    cena_av = numpy.dot(x_rob, av_pov)
    cena_d = sqrt(numpy.dot(x_rob, diagonala))
    cena = cena_av + cena_d
    for i in range(1, len(poti)):
        x_nov = poti[i]
        cena_av_nov = numpy.dot(poti[i], av_pov)
        cena_d_nov = sqrt(numpy.dot(poti[i], diagonala))
        cena_nov = cena_av_nov + cena_d_nov
        if cena_nov < cena:
            x_rob = x_nov
            cena = cena_nov
    return(x_rob, cena)

In [1]:
#Vse kar potrebujemo za izboljsan algoritem
 
[grafi, weights] = min_grid_graf(50 , 5, 1, 100) #ustvari množico negotovosti ter graf
povprecje_w = [i * 1/len(weights) for i in [sum(x) for x in zip(*weights)]] #povprecna cena povezav
diagonalci = numpy.diag(kov_matrika(weights)) #diagonale matrike epsilon
xi = vektorcki_x(grafi, (0, 0), (4, 4)) #vse poti med zacetnim (0,0) in koncnim (V, V) vozliscem

xl = arglexmin(xi, [povprecje_w, diagonalci]) #leva leksikografska resitev
xl_vekt = xl[1]
xr = arglexmin(xi, [diagonalci, povprecje_w]) #desna leksikografska rešitev
xr_vekt = xr[1]
#print(xl_vekt == xr_vekt) #preverimo če sta slučajno enaka

#tocke za zacetek
prva_tocka = xl[0] #vrednosti leve lekikografske rešitve priredi tocko (<xl_vekt, povprecje_w>, <xl_vekt, diagonalci>)
druga_tocka = [prva_tocka[0], tretja_tocka[1]] #(<xl_vekt, povprecje_w>, <xr_vekt, diagonalci>), v nadaljevanju je ne uporabljamo
tretja_tocka = [xr[0][1], xr[0][0]] #vrednosti desne lekikografske rešitve priredi tocko (<xr_vekt, povprecje_w>, <xr_vekt, diagonalci>)
[x_rob, OBJ] = argmin_kor([xl_vekt, xr_vekt], povprecje_w, diagonalci) #x_rob najcenejsa pot izmed vektorjev xl_vekt, xr_vekt, OBJ je cena te poti
P = numpy.dot(povprecje_w, x_rob) #OBJ razclenjen
D = sqrt(numpy.dot(diagonalci, x_rob))  #OBJ razclenjen

#zacetni tocki
a = prva_tocka[0]
b = OBJ - sqrt(tretja_tocka[1]) #presecisce med vodoravnico in krivuljo (OBJ-x)^2


In [10]:
#Izboljsan algoritem
#slaba praksa, xevje in stevec morata biti podana preden zaženemo funkcijo
#rob1 predstavlja levo krajisce intervala, rob2 pa desno. Zacetni rob1 je a, zacetni rob2 je b.

# xevje = []
# stevec = 0
def intervali3(rob1, rob2, OBJ, x, P, D, xi, average_c, d):
    global stevec
    m = (rob1 + rob2) / 2 #srednja tocka intervala
    alfa_m = 2 * (m - OBJ) / (2 * (m - OBJ) - 1) 
    [x_new, pov_x_new, diag_x_new, cena_x_new] = argmin(xi, numpy.dot(alfa_m, average_c), numpy.dot((1-alfa_m), d)) #problem najkrajsih poti obtezen z alfa_m
    stevec = stevec + 1
    if 2 * (m - OBJ) * P - 2 * (m - OBJ) * pov_x_new + diag_x_new <= D: #preverjanje, če pride do izboljsanja
        x = x_new #posodabljanje podatkov
        #print("dodajamo")
        xevje.append(x)
        P = pov_x_new #posodabljanje podatkov
        D = diag_x_new #posodabljanje podatkov
        OBJ = P + D #posodabljanje podatkov
        naklon = 2 * (m - OBJ) #posodabljanje podatkov
        zac_vr = (OBJ - m)^2 - naklon * m #posodabljanje podatkov
        var('z')
        sol = solve(0 == z^2 - (2 * OBJ + naklon) * z +(OBJ^2 - zac_vr), z)
        nsol = [N(s.rhs()) for s in sol]
        #print(nsol)
        if len(nsol) == 1: #krivulja (OBJ-x)^2 in premica y = naklon * x + zac_vr se dotikata
            nsol = [nsol[0], nsol[0]]
        if nsol[0].is_real() == True: #preverja, če prva vrednost ni imaginarna, torej če se krivulja in premica sekata
            #print("prvi interval")
            intervali3(rob1, nsol[0], OBJ, x, P, D, xi, average_c, d)
        if nsol[1].is_real() == True: #enako za drugo vrednost
            #print("drugi interval")
            intervali3(nsol[1], rob2, OBJ, x, P, D, xi, average_c, d)


In [2]:
#poganjanje intervali3, oz. izboljsani algoritem
xevje = []
stevec = 0
intervali3(a, b, OBJ, x_rob, P, D, xi, povprecje_w, diagonalci)
print(xevje)
print(stevec)

In [3]:
#zapisovanje rešitev, ki jih vrne izboljsani algoritem podobno, kot pri naivnem algoritmu

import time
import json

tipi_grafov = [3,4,5,6,7,8,9] #velikosti grafov
casi = [] #povprecni casi komputacij za grafe različnih velikosti
stetja = [] #povprecno število reševanj optimizacijskega problema najkrajših poti
for i in range(0, len(tipi_grafov)):
    print("TIP GRAFA ", tipi_grafov[i])
    vmesni_casi = []
    vmesna_stetja = []
    for j in range(0, 30):
        [grafi, weights] = min_grid_graf(50 , tipi_grafov[i], 1, 100)
        povprecje_w = [i * 1/len(weights) for i in [sum(x) for x in zip(*weights)]]
        diagonalci = numpy.diag(kov_matrika(weights))
        xi = vektorcki_x(grafi, (0, 0), (tipi_grafov[i] - 1, tipi_grafov[i] - 1))
        xl = arglexmin(xi, [povprecje_w, diagonalci])
        xl_vekt = xl[1]
        xr = arglexmin(xi, [diagonalci, povprecje_w])
        xr_vekt = xr[1]
        while xl_vekt == xr_vekt: #če sta leksikografski rešitvi slučajno enaki, algoritem ne vrne smiselnih rezultatov, ponovimo
            [grafi, weights] = min_grid_graf(50 , tipi_grafov[i], 1, 100)
            povprecje_w = [i * 1/len(weights) for i in [sum(x) for x in zip(*weights)]]
            diagonalci = numpy.diag(kov_matrika(weights))
            xi = vektorcki_x(grafi, (0, 0), (tipi_grafov[i] - 1, tipi_grafov[i] - 1))
            xl = arglexmin(xi, [povprecje_w, diagonalci])
            xl_vekt = xl[1]
            xr = arglexmin(xi, [diagonalci, povprecje_w])
            xr_vekt = xr[1]
        prva_tocka = xl[0]
        tretja_tocka = [xr[0][1], xr[0][0]]
        druga_tocka = [prva_tocka[0], tretja_tocka[1]]
        [x_rob, OBJ] = argmin_kor([xl_vekt, xr_vekt], povprecje_w, diagonalci)
        P = numpy.dot(povprecje_w, x_rob)
        D = sqrt(numpy.dot(diagonalci, x_rob))
        a = prva_tocka[0]
        b = OBJ - sqrt(tretja_tocka[1])
        xevje = []
        stevec = 1
        start_time = time.time()
        intervali3(a, b, OBJ, x_rob, P, D, xi, povprecje_w, diagonalci)
        celoten_cas = (time.time() - start_time)
        print("NAREJEN GRAF")
        print("Cas za en graf je", celoten_cas)
        print("Stevec za en graf je ", stevec)
        vmesni_casi.append(celoten_cas)
        vmesna_stetja.append(stevec)
    print("Povprecje vmesnih casov je", mean(vmesni_casi))
    casi.append(mean(vmesni_casi))
    print("Povprecje vmesnih stetji je", mean(vmesna_stetja))
    stetja.append(mean(vmesna_stetja))
    print("USTVARJENI PODATKI ZA TIP")
    
with open("rezultati_improved_na_30.json", 'w') as f:
    f.write(str(casi))
    f.write(str(stetja))